In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

In [ ]:
from keras.applications import ResNet101, ResNet152, ResNet50V2, ResNet101V2, ResNet152V2

In [ ]:
%%capture
!pip install wandb -qqq
import wandb

In [ ]:
from wandb.keras import WandbCallback

In [ ]:
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
############################### caution : terminal commands ###########################################

#-----------------------------empty the datasets forlder before downloading the dataset ------------------------

%cd
%cd .keras/datasets/
!rm -r *

/root
[Errno 2] No such file or directory: '.keras/datasets/'
/root
rm: cannot remove '*': No such file or directory


In [ ]:
############################### download data from given url ###################################3

dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
data_dir = tf.keras.utils.get_file('nature_12K', origin=dataset_url, extract=True)

3816693760/3816687935 [==============================] - 97s 0us/step


In [ ]:
#------------------------------caution : terminal commands --------------------------------------

%cd
%cd .keras/datasets/inaturalist_12K
%mv val test
!mkdir valid

/root
/root/.keras/datasets/inaturalist_12K


In [ ]:
###################### split train data into validation set and training set ##########################

data_folder = '/root/.keras/datasets/inaturalist_12K'
os.chdir(data_folder)

folder_names = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia' ]
for i in range(0,10):
  source = data_folder + "/train/" +folder_names[i]  
  orig_files = os.listdir(source)
  chosen_indexes = random.sample(range(0, len(orig_files)-1), 100)

  destination = data_folder + "/valid/"
  os.chdir(destination)
  os.system('mkdir'+' '+str(folder_names[i]))
  destination = destination + folder_names[i]
  for j in range(0,100):
    shutil.move(   source +"/" + str(   orig_files[   chosen_indexes[j]  ]   )  , destination)


In [ ]:
######################## Correcting the directory location #################################

#data_dir = '/root/.keras/datasets/nature_12K'

data_dir = data_dir.split('/')
data_dir.remove('nature_12K')
data_dir.append('inaturalist_12K')
data_dir = '/'.join(data_dir)


In [ ]:
#----------------------------Taking the train data---------------------------------------------
train_data_dir_path = data_dir + '/train'
train_data_dir = pathlib.Path(train_data_dir_path)

#----------------------------Taking the validation data---------------------------------------
valid_data_dir_path = data_dir + '/valid'
valid_data_dir = pathlib.Path(valid_data_dir_path)

#-----------------------------Taking the test data----------------------------------------
test_data_dir_path = data_dir + '/test'
test_data_dir = pathlib.Path(test_data_dir_path)

In [ ]:
#----------------------------Parameters for the images----------------------------------------
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
########################### converting the image data into dataset ##############################

#--------------------------training dataset ---------------------------------------
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical"
  )

#-------------------------validation dataset ----------------------------------
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  valid_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical"
  )

Found 8999 files belonging to 10 classes.
Found 1000 files belonging to 10 classes.


In [ ]:
######################################## function for using trained model ###############################


def get_desired_model(b_model, input_shape=(180,180,3), num_dense_neurons=1014,activation="relu",activation2="softmax",dropout=None, pooling="max",l_rate=0.0001): 
    
    if b_model=="Xception":
      base_model=Xception
    elif b_model=="InceptionV3":
      base_model=InceptionV3
    elif b_model=="ResNet50":
      base_model=ResNet50
    elif b_model=="MobileNet":
      base_model=MobileNet
    elif b_model == "ResNet101":
      base_model=ResNet101
    elif b_model == "ResNet152":
      base_model=ResNet152
    elif b_model == "ResNet50V2":
      base_model=ResNet50V2
    elif b_model == "ResNet101V2":
      base_model=ResNet101V2
    elif b_model == "ResNet152V2":
      base_model=ResNet152V2
    
    
    

    base = base_model(input_shape=input_shape,include_top=False,weights='imagenet') 
  
  #------------------------------ perform pooling --------------------------------------
    if pooling == "avg": 
        x = GlobalAveragePooling2D()(base.output) 
    elif pooling == "max": 
        x = GlobalMaxPooling2D()(base.output) 

  #---------------------------- Flatten the output  --------------------------------------

    x = layers.Flatten()(base.output)


  #------------------------------- dropout ---------------------------------------------
    if dropout != 0: 
        x = layers.Dropout(dropout)(x) 

  #----------------------------- add fully connected layer ---------------------------------
    x = Dense(num_dense_neurons, activation=activation)(x) 

  #------------------------------- and a logistic layer -------------------------------------
    x = Dense(10, activation=activation2)(x) 

  #----------------------------------- get model ----------------------------------------------
    model = Model(inputs=base.input, outputs=x) 

  #--------------------------------- setting the trainable layers -------------------------------
    for layer in base.layers:
        layer.trainable = False

 #------------------------------------compiling the model -----------------------------------------
    model.compile(loss='categorical_crossentropy', metrics=["accuracy"], optimizer=Adam(l_rate)) 
    
  #-------------------------------------return model -------------------------------------------------
    return model

#============================== end of get_desired_model function ========================================

In [ ]:
#--------------------------- training -------------------------------------------
def train(model, train_ds, val_ds, epochs=5):
  history = model.fit(train_ds, validation_data = val_ds, epochs = epochs, callbacks=[WandbCallback()])
  return history

In [ ]:
#Experiment 3
sweep_config={
              "method":"random",
              'metric' : {
                            'name' : 'val_accuracy',
                            'goal' : 'maximize',
                         },
          "parameters" : {
                            "base_model":{"values":["ResNet50","ResNet101"]},
                            "activation":{"values":['relu']},
                            "activation2":{"values":['softmax']},
                            "input_shape":{"values":[(180,180,3)]},
                            "dropout":{"values":[0,0.1,0.15,0.18]},
                            "num_dense_neuron":{"values":[256,512,1014,1280,2048]},
                            "l_rate":{"values":[0.0001,0.00005]},
                            "epoch": {"values": [5,10]},
                            "pooling":{"values":["max", "avg", "no Pooling"]}
                         }
              }
sweep_id = wandb.sweep(sweep_config, project="pretrained")

Create sweep with ID: ei8cg921
Sweep URL: https://wandb.ai/pranayrajparisha/pretrained/sweeps/ei8cg921


In [ ]:
def run():

  wb = wandb.init()
  config = wb.config

  # sweep parameters
  base_model = config.base_model
  activation = config.activation
  activation2 = config.activation2
  dropout = config.dropout
  input_shape = config.input_shape
  num_dense_neuron = config.num_dense_neuron
  l_rate=config.l_rate
  epoch=config.epoch
  pooling = config.pooling

  
  model = get_desired_model(base_model, input_shape, num_dense_neuron, activation, activation2, dropout, pooling, l_rate)
  history = train(model, train_ds, val_ds, epoch)

  return

In [ ]:
wandb.agent(sweep_id, run)

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ib4v6sm4 with config:
wandb: 	activation: relu
wandb: 	activation2: softmax
wandb: 	base_model: ResNet101
wandb: 	dropout: 0.18
wandb: 	epoch: 10
wandb: 	input_shape: [180, 180, 3]
wandb: 	l_rate: 5e-05
wandb: 	num_dense_neuron: 256
wandb: 	pooling: max
wandb: Currently logged in as: pranayrajparisha (use `wandb login --relogin` to force relogin)


171450368/171446536 [==============================] - 5s 0us/step
Epoch 1/10
282/282 [==============================] - 202s 585ms/step - loss: 2.0407 - accuracy: 0.4586 - val_loss: 1.1449 - val_accuracy: 0.6060
Epoch 2/10
282/282 [==============================] - 173s 605ms/step - loss: 0.6301 - accuracy: 0.7927 - val_loss: 1.1775 - val_accuracy: 0.6560
Epoch 3/10
282/282 [==============================] - 160s 563ms/step - loss: 0.2574 - accuracy: 0.9215 - val_loss: 1.3370 - val_accuracy: 0.6310
Epoch 4/10
282/282 [==============================] - 158s 556ms/step - loss: 0.1028 - accuracy: 0.9716 - val_loss: 1.3860 - val_accuracy: 0.6410
Epoch 5/10
282/282 [==============================] - 161s 564ms/step - loss: 0.0524 - accuracy: 0.9887 - val_loss: 1.5105 - val_accuracy: 0.6510
Epoch 6/10
282/282 [==============================] - 159s 558ms/step - loss: 0.0367 - accuracy: 0.9921 - val_loss: 1.5169 - val_accuracy: 0.6470
Epoch 7/10
282/282 [==============================] - 160

epoch,9
loss,0.01585
accuracy,0.99689
val_loss,1.66766
val_accuracy,0.66
_runtime,1663
_timestamp,1618392927
_step,9
best_val_loss,1.14489
best_epoch,0


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▂▁▁▁▁▁▁▁
accuracy,▁▅▇███████
val_loss,▁▁▄▄▆▆▇█▇█
val_accuracy,▁▆▄▅▆▅▇█▇▇
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: 8e697cuh with config:
wandb: 	activation: relu
wandb: 	activation2: softmax
wandb: 	base_model: ResNet50
wandb: 	dropout: 0
wandb: 	epoch: 10
wandb: 	input_shape: [180, 180, 3]
wandb: 	l_rate: 0.0001
wandb: 	num_dense_neuron: 1280
wandb: 	pooling: no Pooling


...

Epoch 1/5
282/282 [==============================] - 127s 433ms/step - loss: 3.0325 - accuracy: 0.4677 - val_loss: 1.1180 - val_accuracy: 0.6340
Epoch 2/5
282/282 [==============================] - 124s 431ms/step - loss: 0.5459 - accuracy: 0.8238 - val_loss: 1.1536 - val_accuracy: 0.6420
Epoch 3/5
282/282 [==============================] - 122s 428ms/step - loss: 0.1758 - accuracy: 0.9498 - val_loss: 1.3083 - val_accuracy: 0.6440
Epoch 4/5
282/282 [==============================] - 123s 431ms/step - loss: 0.0617 - accuracy: 0.9862 - val_loss: 1.3875 - val_accuracy: 0.6650
Epoch 5/5
282/282 [==============================] - 123s 430ms/step - loss: 0.0249 - accuracy: 0.9974 - val_loss: 1.4508 - val_accuracy: 0.6620


epoch,4
loss,0.02342
accuracy,0.99722
val_loss,1.45082
val_accuracy,0.662
_runtime,631
_timestamp,1618404738
_step,4
best_val_loss,1.11803
best_epoch,0


epoch,▁▃▅▆█
loss,█▃▂▁▁
accuracy,▁▆▇██
val_loss,▁▂▅▇█
val_accuracy,▁▃▃█▇
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: Agent Starting Run: oa1scnwe with config:
wandb: 	activation: relu
wandb: 	activation2: softmax
wandb: 	base_model: ResNet101
wandb: 	dropout: 0.18
wandb: 	epoch: 10
wandb: 	input_shape: [180, 180, 3]
wandb: 	l_rate: 5e-05
wandb: 	num_dense_neuron: 256
wandb: 	pooling: max


Epoch 1/10
282/282 [==============================] - 166s 558ms/step - loss: 1.9991 - accuracy: 0.4560 - val_loss: 1.1243 - val_accuracy: 0.6260
Epoch 2/10
162/282 [================>.............] - ETA: 1:00 - loss: 0.6662 - accuracy: 0.7768